In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 증강

In [ ]:
import os
import random
import numpy as np
import albumentations as A
from PIL import Image
from tqdm import tqdm


# Albumentations 변환 설정
transform = A.Compose([
            A.Blur(always_apply=False, p=0.5, blur_limit=(2, 5)),
            A.ElasticTransform(always_apply=False, p=0.5, alpha=1.0, sigma=50.0, alpha_affine=50.0, interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None, approximate=False, same_dxdy=False),
            A.GaussNoise(always_apply=False, p=0.5, var_limit=(10.0, 50.0), per_channel=True, mean=0.0),
            A.RandomFog(always_apply=False, p=0.5, fog_coef_lower=0.1, fog_coef_upper=0.2, alpha_coef=0.08),
            A.RandomBrightnessContrast(always_apply=False, p=0.5, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
            A.RandomGamma(always_apply=False, p=0.5, gamma_limit=(80, 120), eps=None),
            A.HorizontalFlip(always_apply=False, p=0.5)
        ])

folders = ["꼬임"]

for folder in folders:
    image_names = os.listdir(folder)
    save_folder_path = folder
    print("진행 폴더 :", folder, "기존 이미지 개수 : ",len(os.listdir(folder)))
    for i in tqdm(range(307)):
        image_name = random.choice(image_names)
        image_path = os.path.join(folder, image_name)

        # 이미지 불러오기
        image = Image.open(image_path)

        # Albumentations 적용
        transformed = transform(image=np.array(image))

        # 적용된 이미지 저장
        save_path = os.path.join(save_folder_path, f"aug_{str(i).zfill(3)}.png")
        Image.fromarray(transformed["image"]).save(save_path)
    print("진행 폴더 :", folder, "증강 이미지 개수 : ", len(os.listdir(folder)))
    print("-----------------------------------------")

In [ ]:
import timm

models_list = timm.list_models()


In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="zMJ1XlSC8CveRtknmRR0")
project = rf.workspace("wallpaper-zotss").project("alpaco4th")
dataset = project.version(1).download("folder")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to alpaco4th-1 in folder:: 100%|██████████| 9019/9019 [00:01<00:00, 5172.16it/s]


In [ ]:
!pip install transformers
!pip install torch
!pip install torchvision
!pip install tqdm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import os
import numpy as np
import torch
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정

In [ ]:
import glob
all_img_list_1 = glob.glob('/content/alpaco4th-1/train/*/*')
all_img_list_2 = glob.glob('/content/alpaco4th-1/test/*/*')
all_img_list_3 = glob.glob('/content/alpaco4th-1/valid/*/*')

In [ ]:
all_img_list = all_img_list_1 + all_img_list_2 + all_img_list_3

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[4])

In [ ]:
len(all_img_list)

8960

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[4])

In [ ]:
from sklearn.model_selection import train_test_split
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=42)

In [ ]:

le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [ ]:
train

,img_path,label
5540,/content/alpaco4th-1/train/10/190_png.rf.79efa...,10
4437,/content/alpaco4th-1/train/00/aug_178_png.rf.4...,0
6801,/content/alpaco4th-1/test/07/52_png.rf.7a1962e...,7
4456,/content/alpaco4th-1/train/00/aug_220_png.rf.2...,0
4658,/content/alpaco4th-1/train/12/aug_244_png.rf.2...,12
...,...,...
1967,/content/alpaco4th-1/train/01/aug_058_png.rf.1...,1
3426,/content/alpaco4th-1/train/04/aug_066_png.rf.8...,4
5928,/content/alpaco4th-1/train/02/aug_136_png.rf.1...,2
7959,/content/alpaco4th-1/valid/17/493_png.rf.cbcc1...,17


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)



In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':3,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
to_transform = A.Compose([
                        A.Resize(224,224),
                        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                        ToTensorV2()
                            ])
# 학습 데이터와 검증 데이터를 로드하고 전처리

train_dataset = CustomDataset(train['img_path'].values, train['label'].values, to_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, to_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

In [ ]:
import os
import torch
import timm
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
# 모델 로드
import timm

model = timm.create_model("vit_base_resnet50_224_in21k", pretrained=True,num_classes=18)

# GPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 손실 함수와 옵티마이저
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=3e-5, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 변수 설정
n_splits = 10

# K-Fold 설정
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=CFG['SEED'])

# 결과 저장을 위한 변수 초기화
oof_preds = np.zeros((len(df), 18))
best_models = []

# K-Fold 학습 및 검증
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['label'])):
    print(f'Fold: {fold + 1}')
    
    train_data = df.iloc[train_idx]
    val_data = df.iloc[val_idx]
    
    le = preprocessing.LabelEncoder()
    train_data['label'] = le.fit_transform(train_data['label'])
    val_data['label'] = le.transform(val_data['label'])
    
    train_dataset = CustomDataset(train_data['img_path'].values, train_data['label'].values, to_transform)
    train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=4)
    
    val_dataset = CustomDataset(val_data['img_path'].values, val_data['label'].values, to_transform)
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4)
    
    model = timm.create_model("vit_base_resnet50_224_in21k", pretrained=True, num_classes=18)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    criterion = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=3e-5, weight_decay=0.01)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

    best_model = train(model, optimizer, train_loader, val_loader, scheduler, device)
    best_models.append(best_model)
    
    with torch.no_grad():
        for idx, (imgs, _) in enumerate(val_dataset):
            img = imgs.unsqueeze(0).to(device)
            pred = best_model(img)
            oof_preds[val_idx[idx]] = pred.detach().cpu().numpy()


Fold: 1


100%|██████████| 28/28 [00:14<00:00,  2.00it/s]

Epoch [1], Train Loss : [0.63023] Val Loss : [0.18518] Val Weighted F1 Score : [0.95163]



100%|██████████| 28/28 [00:14<00:00,  1.98it/s]

Epoch [2], Train Loss : [0.12844] Val Loss : [0.17217] Val Weighted F1 Score : [0.95191]


...

Epoch [2], Train Loss : [0.12457] Val Loss : [0.19034] Val Weighted F1 Score : [0.94589]



100%|██████████| 28/28 [00:14<00:00,  1.99it/s]


Epoch [3], Train Loss : [0.10199] Val Loss : [0.21012] Val Weighted F1 Score : [0.93249]
Fold: 8


  0%|          | 1/252 [00:02<09:40,  2.31s/it]


OutOfMemoryError: ignored

In [ ]:
best_models

[VisionTransformer(
   (patch_embed): HybridEmbed(
     (backbone): ResNetV2(
       (stem): Sequential(
         (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
         (norm): GroupNormAct(
           32, 64, eps=1e-05, affine=True
           (drop): Identity()
           (act): ReLU(inplace=True)
         )
         (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
       )
       (stages): Sequential(
         (0): ResNetStage(
           (blocks): Sequential(
             (0): Bottleneck(
               (downsample): DownsampleConv(
                 (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                 (norm): GroupNormAct(
                   32, 256, eps=1e-05, affine=True
                   (drop): Identity()
                   (act): Identity()
                 )
               )
               (conv1): StdConv2dSame(64, 64, kernel_size=(1, 1), stride=(

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/testfile/test.csv')
test["img_path"] = test["img_path"].str.replace("./test", "/content/drive/MyDrive/testfile/test")
test_dataset = CustomDataset(test['img_path'].values, None, to_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4)


In [ ]:
import torch
torch.cuda.empty_cache()

IndentationError: ignored

In [ ]:
import gc
gc.collect()

268

In [ ]:
def ensemble_predict(models, test_loader, device):
    preds = []
    
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.float().to(device)
            temp_preds = []
            
            for model in models:
                model.eval()
                output = model(imgs)
                temp_preds.append(output.detach().cpu().numpy())
                
            avg_preds = np.mean(temp_preds, axis=0)
            preds.extend(avg_preds.argmax(1))
            
    return preds

final_preds = ensemble_predict(best_models, test_loader, device)


100%|██████████| 25/25 [01:25<00:00,  3.42s/it]


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/testfile/sample_submission.csv')

In [ ]:
submit['label'] = final_preds

In [ ]:
submit.to_csv('/content/meta2.csv', index=False)

In [ ]:

import csv

# 딕셔너리 생성
label_dict = {
    0: '가구수정',
    1: '걸레받이수정',
    2: '곰팡이',
    3: '꼬임',
    4: '녹오염',
    5: '들뜸',
    6: '면불량',
    7: '몰딩수정',
    8: '반점',
    9: '석고수정',
    10: '오염',
    11: '오타공',
    12: '울음',
    13: '이음부불량',
    14: '창틀문틀수정',
    15: '터짐',
    16: '피스',
    17: '훼손'
}

# CSV 파일 열기
with open('/content/meta2.csv', mode='r') as file:
    reader = csv.reader(file)
    submission = list(reader)

# 레이블 번호를 한글로 변경
for i in range(1, len(submission)):
    submission[i][1] = label_dict[int(submission[i][1])]

# 새로운 CSV 파일로 저장
with open('/content/chanho.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(submission)